In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from datetime import datetime as dt, timedelta
from pandas_profiling import ProfileReport
plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns; sns.set()  # for plot styling
from sklearn.pipeline import Pipeline
import plotly.graph_objects as go
import datetime
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
display(HTML("<style>.container { width:100% !important; }</style>"))
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error,r2_score
import math
from sklearn.model_selection import train_test_split
import xlsxwriter
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
from openpyxl import load_workbook
# writer = pd.ExcelWriter('output.xlsx', engine = 'xlsxwriter')
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
SCOPE = ['https://www.googleapis.com/auth/drive']
CREDENTIALS = ServiceAccountCredentials.from_json_keyfile_name('expanded-bonbon-239019-ed390949c71b.json', SCOPE)

In [2]:
# oc = pd.DataFrame()
# for zone in zv_zone.loc[zv_zone["Zona në sistem"] != ""]["Zona në sistem"]:
    
# #     Ratios = process.extract(r, path_to_map['Column'].dropna().unique()) # Extracts all matches for the given string
#     highest = process.extractOne(zone, elite.loc[elite.century_zone == "Not mapped"].zone.drop_duplicates() ) # Extracts the highest match for the given string
#     row = {
#         'elite': zone,
#         'century': highest[0],
#         'Percentage': highest[1]
#     }
#     oc = oc.append(row, ignore_index=True)
    
# elite_zones = oc.rename(columns={
#     'String': 'century'
# })[['century', 'elite', 'Percentage']]

In [2]:
xls = pd.ExcelFile('datasets/Apartments2021.xlsx')
century = pd.read_excel(xls, 'century21')
elite = pd.read_excel(xls, 'elite')
futurehome = pd.read_excel(xls, 'futurehome')
mei = pd.read_excel(xls, 'mei')
properties = pd.read_excel(xls, 'All agencies')

In [3]:
#Remove records with Not Mapped zones
elite = elite.loc[elite.century_zone != 'Not mapped']
futurehome = futurehome.loc[futurehome.century_zone != 'Not mapped']
mei = mei.loc[mei.century_zone != 'Not mapped']
properties = properties.loc[properties.century_zone != 'Not mapped']

# Pandas Profiling

In [5]:
# profile = ProfileReport(properties, title="Pandas Profiling Report")
# profile.to_file('properties')
# profile = ProfileReport(elite, title="Pandas Profiling Report")
# profile.to_file('elite')
# profile = ProfileReport(century, title="Pandas Profiling Report")
# profile.to_file('century')
# profile = ProfileReport(futurehome, title="Pandas Profiling Report")
# profile.to_file('futurehome')
# profile = ProfileReport(mei, title="Pandas Profiling Report")
# profile.to_file('mei')

# Full Dataset Cleaning

In [4]:
# properties.groupby("property_status").count()
#If status NaN make status Not Applicable
properties.property_status.loc[properties.property_status.isna()] = 'Not Applicable'
#Keep only the statuses New & Not Applicable & Under Construction & Used
properties = properties.loc[(properties.property_status == 'New') | (properties.property_status == 'Not Applicable') | (properties.property_status == 'Under Construction') | (properties.property_status == 'Used')]

#Keep only the properties in Tirane
properties = properties.loc[properties.division == 'Tirana']

# Zone cleaning
properties.century_zone.loc[properties.century_zone == '21 dhjetori'] = '21 Dhjetori'
properties.century_zone.loc[properties.century_zone == 'Ali Demi Materniteti i Ri'] = 'Ali Demi'
properties.century_zone.loc[properties.century_zone == 'Ish Blloku'] = 'Blloku'
properties = properties.sort_values(by="century_zone")

# Unaza e Re

In [7]:
# #create new feature for rooms like 2+1, 3+1 
# for index in unaza_e_re.index:
#     if unaza_e_re.loc[index].other_rooms != 0:
#         unaza_e_re["rooms"].loc[index] = str(unaza_e_re.bedrooms.loc[index]) + "+" + str(unaza_e_re.other_rooms.loc[index])
    
# unaza_e_re

## Cleaning dataset

In [35]:
unaza_e_re = properties.loc[properties.century_zone == 'Unaza e Re']
unaza_e_re.columns
unaza_e_re = unaza_e_re[['propertiesid','property_status', 'price', 'interior_area', 'gros_area', 'bedrooms', 'bathrooms','other_rooms']]
unaza_e_re.bedrooms.fillna(0, inplace=True)
unaza_e_re.other_rooms.fillna(0, inplace=True)
unaza_e_re = unaza_e_re.loc[unaza_e_re.index != 12968]
unaza_e_re = unaza_e_re.loc[unaza_e_re.index != 4174]
#if gross area 0 or smaller interior area then replace with interior area and vice versa
unaza_e_re["temp"] = 0
unaza_e_re.temp.loc[(unaza_e_re.interior_area > unaza_e_re.gros_area)] = unaza_e_re.interior_area.loc[(unaza_e_re.interior_area > unaza_e_re.gros_area)]
unaza_e_re.interior_area.loc[(unaza_e_re.interior_area > unaza_e_re.gros_area)] = unaza_e_re.gros_area.loc[(unaza_e_re.interior_area > unaza_e_re.gros_area)]
unaza_e_re.gros_area.loc[(unaza_e_re.interior_area > unaza_e_re.gros_area)] = unaza_e_re.temp.loc[(unaza_e_re.interior_area > unaza_e_re.gros_area)]
unaza_e_re.gros_area.loc[unaza_e_re.gros_area == 0] = unaza_e_re.interior_area.loc[unaza_e_re.gros_area == 0]
unaza_e_re.interior_area.loc[unaza_e_re.interior_area == 0] = unaza_e_re.gros_area.loc[unaza_e_re.interior_area == 0]

#remove record with no rooms specified
unaza_e_re = unaza_e_re.loc[(unaza_e_re.bedrooms != 0) | (unaza_e_re.other_rooms != 0)]
unaza_e_re = unaza_e_re.loc[unaza_e_re.price > 280]
unaza_e_re = unaza_e_re.loc[unaza_e_re.gros_area > 1]
#create new feature combining(bedrooms and other_rooms)
unaza_e_re["rooms"] = 0
for index in unaza_e_re.index:
    unaza_e_re["rooms"].loc[index] = unaza_e_re.loc[index]["bedrooms"] + unaza_e_re.loc[index]["other_rooms"]

unaza_e_re.temp.loc[unaza_e_re.bathrooms > unaza_e_re.rooms] = unaza_e_re.bathrooms.loc[unaza_e_re.bathrooms > unaza_e_re.rooms]
unaza_e_re.bathrooms.loc[unaza_e_re.bathrooms > unaza_e_re.rooms] = unaza_e_re.rooms.loc[unaza_e_re.bathrooms > unaza_e_re.rooms]
unaza_e_re.rooms.loc[unaza_e_re.bathrooms > unaza_e_re.rooms] = unaza_e_re.temp.loc[unaza_e_re.bathrooms > unaza_e_re.rooms]

#drop bedrooms/other_rooms features
unaza_e_re.drop('bedrooms',1, inplace=True)
unaza_e_re.drop('other_rooms',1, inplace=True)
unaza_e_re.drop('temp',1, inplace=True)
unaza_e_re = unaza_e_re.loc[unaza_e_re.bathrooms != 21]
unaza_e_re.bathrooms.loc[unaza_e_re.bathrooms == 0] = 1
unaza_e_re = unaza_e_re.loc[unaza_e_re.rooms < 3]
# profile = ProfileReport(unaza_e_re, title="Pandas Profiling Report")
# profile.to_file('unaza_e_re')
unaza_e_re_nl = unaza_e_re.copy()
#Linear Relationship between gross area and price
unaza_e_re = unaza_e_re.loc[(unaza_e_re['interior_area'] < 140) & (unaza_e_re['price'] < 115000) & (unaza_e_re['price'] > 40000)]

In [36]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Scatter(x=unaza_e_re_nl["price"], y=unaza_e_re_nl["interior_area"], mode="markers+text"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=unaza_e_re["price"], y=unaza_e_re["interior_area"], mode="markers+text"),
    row=2, col=1
)

fig.update_layout(title_text="Linear Relationship Created")
fig.show()

In [34]:
fig = px.scatter(x=unaza_e_re["interior_area"], y=unaza_e_re["bathrooms"])
fig.show()

# Correlation of Features

In [11]:
# unaza_e_re_corr = unaza_e_re_new.corr()
# sns.heatmap(unaza_e_re_corr, xticklabels=unaza_e_re_corr.columns, yticklabels=unaza_e_re_corr.columns, annot=True, cmap="Blues" )

## Handling categorical data

In [12]:
# property_status = pd.get_dummies(unaza_e_re["property_status"])
# unaza_e_re = unaza_e_re.merge(property_status, how='inner', left_index=True, right_index=True)
# unaza_e_re.drop('property_status',1,inplace=True)

In [40]:
%store unaza_e_re
unaza_e_re_new = unaza_e_re[['propertiesid','price', 'interior_area', 'gros_area', 'bathrooms', 'rooms']] 
%store unaza_e_re_new

Stored 'unaza_e_re' (DataFrame)
Stored 'unaza_e_re_new' (DataFrame)


In [14]:
#TO-D0 scaling of dataframe and cross validation using XGboost and RandomForest

# Fresku

## Cleaning dataset

In [57]:
fresku = properties.loc[properties.century_zone == 'Fresku']

fresku = fresku[['propertiesid','property_status', 'price', 'interior_area', 'gros_area', 'bedrooms', 'bathrooms','other_rooms']]
fresku.bedrooms.fillna(0, inplace=True)
fresku.other_rooms.fillna(0, inplace=True)
#if gross area 0 then replace with interior area and vice versa
fresku.gros_area.loc[fresku.gros_area == 0] = fresku.interior_area.loc[fresku.gros_area == 0]
fresku.interior_area.loc[fresku.interior_area == 0] = fresku.gros_area.loc[fresku.interior_area == 0]
#remove record with no rooms specified
fresku = fresku.loc[(fresku.bedrooms != 0) | (fresku.other_rooms != 0)]
fresku = fresku.loc[fresku.price > 18000]
fresku = fresku.loc[fresku.gros_area > 1]
fresku = fresku.loc[fresku.interior_area < 240]
#create new feature combining(bedrooms and other_rooms)
fresku["rooms"] = 0
fresku["temp"] = 0
for index in fresku.index:
    fresku["rooms"].loc[index] = fresku.loc[index]["bedrooms"] + fresku.loc[index]["other_rooms"]

fresku.temp.loc[fresku.bathrooms > fresku.rooms] = fresku.bathrooms.loc[fresku.bathrooms > fresku.rooms]
fresku.bathrooms.loc[fresku.bathrooms > fresku.rooms] = fresku.rooms.loc[fresku.bathrooms > fresku.rooms]
fresku.rooms.loc[fresku.bathrooms > fresku.rooms] = fresku.temp.loc[fresku.bathrooms > fresku.rooms]

#drop bedrooms/other_rooms features

fresku = fresku.loc[fresku.bathrooms < 3]
fresku.bathrooms.loc[fresku.bathrooms == 0] = 1
fresku = fresku.loc[fresku.rooms < 6]
    
#drop bedrooms/other_rooms features
fresku.drop('bedrooms',1, inplace=True)
fresku.drop('other_rooms',1, inplace=True)
fresku.drop('temp',1, inplace=True)
# profile = ProfileReport(unaza_e_re, title="Pandas Profiling Report")
# profile.to_file('unaza_e_re')
fresku_nl = fresku.copy()
#Linear Relationship between gross area and price
fresku = fresku.loc[(fresku['interior_area'] < 165)& (fresku['interior_area'] >= 49) & (fresku['price'] < 175000) & (fresku['price'] < 155000)] 
# fresku = fresku.loc[(fresku['gros_area'] < 313) & (fresku['gros_area'] != 198)  & (fresku['gros_area'] != 101) & (fresku['gros_area'] != 263)& (fresku['gros_area'] > 35) & (fresku['price'] < 175000)]

## Linear Relationship Transformed

In [58]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Scatter(x=fresku_nl["interior_area"], y=fresku_nl["price"], mode="markers+text"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=fresku["interior_area"], y=fresku["price"], mode="markers+text"),
    row=2, col=1
)

fig.update_layout(title_text="Linear Relationship Created")
fig.show()

In [59]:
fig = px.scatter(x=fresku["interior_area"], y=fresku["bathrooms"])
fig.show()

## Handling categorical data

In [18]:
# fresku_new = fresku[['propertiesid','price', 'interior_area', 'gros_area', 'bathrooms', 'rooms']]

In [29]:
# property_status = pd.get_dummies(fresku["property_status"])
# fresku = fresku.merge(property_status, how='inner', left_index=True, right_index=True)
# fresku.drop('property_status',1,inplace=True)
# fresku

In [60]:
%store fresku
fresku_new = fresku.copy()
%store fresku_new

Stored 'fresku' (DataFrame)
Stored 'fresku_new' (DataFrame)
